In [1]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau

# telegram
import telegram
from callbacks_addons import TelegramCallback

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time

Using TensorFlow backend.


In [2]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.2

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [3]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

        tensor =  tf.convert_to_tensor(self.settings_str)
        summary = tf.summary.text("Run_Settings", tensor)

        with  tf.Session() as sess:
            s = sess.run(summary)
            self.writer.add_summary(s)

In [4]:
smartphones = ["N5X"]
window_size = 25
constant_pixels = 0.06
HDF5_PATH = str(Path.home())+"/data/hdf/"+smartphones[0]+"-win"+str(window_size)+".hdf"

# H5DF files contains 3 members: labels, segments, groups

In [5]:
hdf = h5py.File(HDF5_PATH, "r")

train_x = hdf["train/sensors"]
train_y = hdf["train/labels"]

test_x = hdf["test/sensors"]
test_y = hdf["test/labels"]

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(1161002, 25, 18, 1) (1161002, 2) (501160, 25, 18, 1) (501160, 2)


In [6]:
pSensors = hdf["train" + "/sensors"]

In [7]:
pSensors[0].shape

(25, 18, 1)

In [8]:
def myGenerator(set_name, batch_size):
    """
    This generator returns images
    """
    hdf = h5py.File(HDF5_PATH, "r")

    pSensors = hdf[set_name + "/sensors"]
    pLabels = hdf[set_name + "/labels"]

    len_train = pSensors.shape[0]
    
    randomBatchOrder = np.arange(len_train // batch_size)

    while True:
        np.random.shuffle(randomBatchOrder) 
        
        for i in range(len_train // batch_size):
            idx = randomBatchOrder[i]
            shuffled = shuffle(pSensors[idx * batch_size: (idx+1) * batch_size], pLabels[idx * batch_size: (idx+1) * batch_size])
            yield shuffled[0].reshape(-1, train_x.shape[1], train_x.shape[2]), shuffled[1].reshape(-1, train_y.shape[1])
            
# (n_frames, n_values, 1); labels

In [9]:
## ConvLSTM net hyperparameters
numFilters1 = 128 # number of filters in first Conv2D layer
numFilters2 = 64 # number of filters in second Conv2D layer
# kernal size of the Conv2D layer
kernalSize1 = (3, 3)
# max pooling window size
poolingWindowSz = 2
# number of filters in fully connected layers
numNueronsFCL1 = 512
numNueronsFCL2 = 128
# number of epochs
epochs = 9999
# batchsize
batch_size = 200
# number of total clases
numClasses = train_y.shape[1]
# dropout ratio for dropout layer
dropOutRatio = 0.5

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'), input_shape=(None, train_x.shape[2], 1))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(LSTM(75, return_sequences=True))
model.add(LSTM(75))
model.add(Dense(2))

model = Sequential()
model.add(LSTM(75, input_shape=(None, train_x.shape[2]), return_sequences=True))
model.add(LSTM(75))
model.add(Dense(2))

In [10]:
train_x.shape[1], train_x.shape[2]

(25, 18)

In [ ]:
with tf.device('/device:GPU:1'):
    tf.get_default_graph()
    model = Sequential()
    model.add(LSTM(75, input_shape=(None, train_x.shape[2]), return_sequences=True))
    model.add(LSTM(75))
    model.add(Dense(2))

    ## Compiling the model to generate a model
    def eucInMM(y_true, y_pred):
        return (K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))) * constant_pixels
    def euc(y_true, y_pred):
        return K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))
    def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_true - y_pred)))

    optimizer = optimizers.Adam(lr = 0.001, decay=1e-6)
    #optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
    model.compile(loss=rmse, optimizer=optimizer, metrics=[eucInMM ,rmse, euc])

    # Broadcast progress to the tensorboard.
    readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
    file_name = "IMU_" + smartphones[0] + "_" + readable_timestamp
    tensorboardFolder = "/srv/share/tensorboardfiles/" + file_name

    config = ""
    for layer in model.layers:
        config += str(layer.output).split('\"')[1].split("/")[0] + str(layer.output_shape) + "\n\n"
    config += "Kernel Size: " + str(kernalSize1) + "\n\n"
    config += "Batch Size: " + str(batch_size) + "\n\n"
    config += "Num of Epochs: " + str(epochs) + "\n\n"

    print(file_name)
    model.summary()

    callbacks = [
        ModelCheckpoint(str(Path.home()) + "/data/models/"+ file_name + ".{epoch:02d}-{val_loss:.2f}.h5",
                        monitor='val_eucInMM',
                        verbose=0,
                        save_best_only=False,
                        save_weights_only=False
                       ),
        #LoggingTensorBoard(settings_str_to_log=config,
        #                   log_dir=tensorboardFolder,
        #                   histogram_freq=0,
        #                   write_graph=True,
        #                   write_images=True,
        #                   update_freq='epoch'
        #                  ),
        ReduceLROnPlateau(monitor='val_loss',
                          patience=10,
                          verbose=1,
                          factor=0.9,
                          min_lr=0.00001),
        TelegramCallback(
            {
                'token': "707020808:AAHJ22G0diUExRP4JUITk1VKuPtBPUVQ-J4",
                'telegram_id': -320890356,
            }
        )
    ]

    history = model.fit_generator(myGenerator("train", batch_size),
                                  steps_per_epoch=len(train_x) // batch_size,
                                  epochs=epochs,
                                  verbose=1,
                                  validation_data=myGenerator("test", batch_size),
                                  validation_steps=len(test_x) // batch_size,
                                  callbacks=callbacks
                                 )

    score = model.evaluate(np.expand_dims(test_x,1),np.expand_dims(test_y,1),verbose=2)
    print('%s: %.2f' % (model.metrics_names[1], score[1]))
    print('Baseline ConvLSTM Error: %.2f' %(1 - score[1]))

IMU_N5X_20190110_223547
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 75)          28200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 75)                45300     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 152       
Total params: 73,652
Trainable params: 73,652
Non-trainable params: 0
_________________________________________________________________
Epoch 1/9999
2992/5805 [==============>...............] - ETA: 5:10 - loss: 829.2198 - eucInMM: 65.5998 - rmse: 829.2198 - euc: 1093.3299